# App Stat Exam2023    -    **Exam Number: 100**

## Importing libraries:

In [1]:
#Other good imports                        
from numpy.linalg import matrix_power                                                                       
from iminuit import Minuit                                                                 
from scipy.integrate import quad
import sympy as sp
from sympy import integrate , print_latex
import pandas as pd
from IPython.display import display


#Packages needed for the external functions
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.metrics import roc_curve, auc
from iminuit.util import make_func_code
from iminuit import describe 


# import sequence of functions from external file
import sys 
# sys.path.append('../External_Functions')
from MyExternalFunctionsForExam import *

Matplotlib_settings()

## plt.savefig('.pdf')                     #Save the figure as pdf command
## np.loadtxt('.txt')                      OPTIONAL: skiprows=#, usecols=(#,#,#,#), comments='#', delimiter=',' 
## - alternative: np.genfromtxt('.txt', skip_header=1, skip_footer=4)

## 1) Distributions and probabilities

### 1.1) 

In [21]:
#Two gaussian distributions with mu=50 and sigma=20
mu = 50
sigma = 20


#1.1.1)
Frac_between_55_and_65 = stats.norm.cdf(65,mu,sigma)-stats.norm.cdf(55,mu,sigma)
print(Frac_between_55_and_65)

#1.1.2) 
#Here we do get the actual real mu and sigma values, so no loss in ndof for estimating. 
error_on_mean = sigma/np.sqrt(120)
print(error_on_mean)

#1.1.3)


# Correlation coefficient
rho = 0.60

# Z-score for a score of 60
z = (60 - mu) / sigma

new_mu = rho*z
new_sigma = np.sqrt(1 - rho**2)

# Bivariate normal cumulative distribution function
p = stats.norm.cdf(z, new_mu, new_sigma)

# Probability of getting a score above 60 in both tests
prob = p**2
print(prob)


0.17466632194020804
1.8257418583505538
0.3584492644127471


### 1.2)

In [27]:
win_rate = 0.4 
min_wins = 11
N_games = 20 

chance_of_winning = stats.binom.sf(min_wins-1, N_games, win_rate)
print(chance_of_winning)

0.12752124614721674


## 2) Error Propagation

### 2.1) 

### 2.2) 

## 3) Simulation / Monte Carlo 

### 3.1)

## 4) Statistical Tests

### 4.1)

## 5) Fitting data

### 5.1)

### 5.2)